In [6]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u

c = const.c.value
G = const.G.value
hbar = const.hbar.value

In [7]:
# def U_eff(l, μ, r):
#     return l**2/(2*μ*r**2) - G*m_1*m_2/r
 
# def findv0(E, l, μ, r):
#     v_0 = np.abs(np.sqrt(2*(E - U_eff(l, μ, r))/μ))
#     return v_0


In [8]:
# def findμ(m_1, m_2):
#     μ = m_1*m_2/(m_1 + m_2)
#     return μ

# def findl(μ, r, v):
#     return μ*np.cross(r,v)

def rel_len(x_1, x_2):
    return np.sqrt((x_1[0] - x_2[0])**2 + (x_1[1] - x_2[1])**2)

# def findE_eff(x1_0, x2_0, v1_0, v2_0, m1, m2):
#     r_vec = x1_0 - x2_0
#     r = rel_len(x1_0, x2_0)
#     v_vec = v1_0 - v2_0
#     v = rel_len(v1_0, v2_0)
#     μ = μ(m1, m2)
#     return μ*v**2/2 + l(μ,r_vec,v_vec)**2/(2*μ*r**2) - G*m1*m2/r

In [11]:
def accelerationN(x_1, x_2, m_1, m_2):
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    n = r_vec/r

    a_1 = - G*m_2*n/r**2
    a_2 =   G*m_1*n/r**2
    return np.array([a_1, a_2])

def accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2):
    r_vec = x_1 - x_2
    r = rel_len(x_1, x_2)
    v_vec = v_1 - v_2
    n = r_vec/r

    a_1 = - G*m_2/r**2*n
    + 1/c**2*(5*G**2*m_1*m_2/r**3 + 4*G**2*m_2**2/r**3 + G*m_2/r**2*(3/2*np.dot(n, v_2)**2 - np.dot(v_1, v_1) + 4*np.dot(v_1, v_2) - 2*np.dot(v_2, v_2)))*n
    + G*m_2/r**2 *(4*np.dot(n, v_1) - 3*np.dot(n, v_2))*v_vec

    a_2 = - G*m_1/r**2*(-n)
    + 1/c**2*(5*G**2*m_2*m_1/r**3 + 4*G**2*m_1**2/r**3 + G*m_1/r**2*(3/2*np.dot((-n), v_1)**2 - np.dot(v_2, v_2) + 4*np.dot(v_2, v_1) - 2*np.dot(v_1, v_1)))*(-n)
    + G*m_1/r**2 *(4*np.dot((-n), v_2) - 3*np.dot((-n), v_1))*(-v_vec)
    return np.array([a_1, a_2])

def boost(v_1, v_2, a_1, a_2, dt):
    v_1 += a_1*dt
    v_2 += a_2*dt
    return np.array([v_1, v_2])

def move(x_1, x_2, v_1, v_2, dt):
    x_1 += v_1*dt
    x_2 += v_2*dt
    return np.array([x_1, x_2])

def findCM(x_1, x_2, m_1, m_2):
    M = m_1 + m_2
    r_cm = np.array([(m_1*x_1[0] + m_2*x_2[0])/M, (m_1*x_1[1] + m_2*x_2[1])/M])
    return r_cm

In [18]:
def run_model(s, t_max, dt, EIH):
    x_1, x_2, v_1, v_2, m_1, m_2 = s
    
    pos1 = np.zeros((2, int(t_max/dt)))
    pos2 = np.zeros((2, int(t_max/dt)))
    pos_CM = np.zeros((2, int(t_max/dt)))

    for i in range(int(t_max/dt)):
        pos1[:,i] = x_1
        pos2[:,i] = x_2
        pos_CM[:,i] = findCM(x_1, x_2, m_1, m_2)
        if EIH == True:
            a_1, a_2 = accelerationEIH(x_1, x_2, v_1, v_2, m_1, m_2)
        else:
            a_1, a_2 = accelerationN(x_1, x_2, m_1, m_2)
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)
    return pos1, pos2, pos_CM




In [15]:
def findMaxPos(pos):
    xmax, xmin, ymax, ymin = max(pos[0]), min(pos[0]), max(pos[1]), min(pos[1])
    return xmax, xmin, ymax, ymin

def plotLims(pos):
    start_pos = np.array([pos[0][0], pos[1][0]])
    end_pos = np.array([pos[0][-1], pos[1][-1]])

    xmax, xmin, ymax, ymin = findMaxPos(pos)
    max_pos = max(max(abs(start_pos)),max(abs(end_pos)))

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

def plotLimsTwoBody(pos1, pos2):
    start_pos1 = np.array([pos1[0][0], pos1[1][0]])
    end_pos1 = np.array([pos1[0][-1], pos1[1][-1]])
    start_pos2 = np.array([pos2[0][0], pos2[1][0]])
    end_pos2 = np.array([pos2[0][-1], pos2[1][-1]])

    x1max, x1min, y1max, y1min = findMaxPos(pos1)
    x2max, x2min, y2max, y2min = findMaxPos(pos2)

    xmax = max(x1max, x2max)
    xmin = min(x1min, x2min)
    ymax = max(y1max, y2max)
    ymin = min(y1min, y2min)

    max_pos1 = max(max(abs(start_pos1)),max(abs(end_pos1)))
    max_pos2 = max(max(abs(start_pos2)),max(abs(end_pos2)))
    max_pos = max(max_pos1, max_pos2)

    xlim = (xmin-0.1*max_pos, xmax+0.1*max_pos)
    ylim = (ymin-0.1*max_pos, ymax+0.1*max_pos)
    return xlim, ylim

(1.3465909215001088e+30, -2.6931818430002176e+29)